# 텍스트 분석

***

**감성분석(Sentiment Analysis)**은 문서의 주관적인 감성/의견/감정/기분 등을 파악하는 방법으로 SNS, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용된다. 주관적인 생각으로는 text classification과 동일한 개념이라고 생각한다. 하지만 감성분석은 크게 **지도학습** 방법과 **비지도 학습** 방법이 있다.

### 지도학습 기반 감성분석


In [57]:
import pandas as pd

In [58]:
train = pd.read_csv("labeledTrainData.tsv", header = 0, sep = '\t', quoting = 3)

In [59]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [60]:
train['sentiment'].value_counts()

0    12500
1    12500
Name: sentiment, dtype: int64

In [61]:
test = pd.read_csv('TestData.tsv', header = 0, sep = '\t', quoting = 3)

In [62]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [63]:
import re

In [64]:
train['review'] = train['review'].str.replace('<br />', ' ')
test['review'] = test['review'].str.replace('<br />', ' ')

In [65]:
train['review'] = train['review'].apply(lambda x : re.sub(r'[^a-zA-Z]', ' ', x))
test['review'] = test['review'].apply(lambda x : re.sub(r'[^a-zA-Z]', ' ', x))

In [66]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [67]:
y = train['sentiment']

In [68]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

In [69]:
gbc = GradientBoostingClassifier(random_state = 42)
lgbm = LGBMClassifier(random_state = 42)
xgb = XGBClassifier(random_state = 42)

In [70]:
def get_model_proba(model, train, test) :
    print(f'{model.__class__.__name__} Train & Predict Start!\n')
    model_pred = np.zeros((len(cnt_test)))
    for i, idx in enumerate(zip(skf.split(train, y))) :
        tr_x, tr_y = train[idx[0][0]], y.iloc[idx[0][0]]
        val_x, val_y = train[idx[0][1]], y.iloc[idx[0][1]]
    
        model.fit(tr_x, tr_y)
        
        val_pred = model.predict_proba(val_x)[:, 1]
        val_cls = [1 if p > 0.5 else 0 for p in val_pred]
        
        acc = accuracy_score(val_y, val_cls)
        roc_auc = roc_auc_score(val_y, val_pred)
        
        print(f'{i + 1} Fold 정확도 = {acc} / roc_auc = {roc_auc}\n')
        
        model_pred += (model.predict_proba(test)[:, 1] / 10)
        
    return model_pred

#### CountVectorizer 적용

In [71]:
cnt_vect = CountVectorizer(max_features = 3000, ngram_range = (1, 2), stop_words = 'english')

In [ ]:
cnt_train = cnt_vect.fit_transform(train['review']).todense()

In [ ]:
cnt_test = cnt_vect.transform(test['review']).todense()

In [ ]:
xgb_pred = get_model_proba(xgb, cnt_train, cnt_test)
lgbm_pred = get_model_proba(lgbm, cnt_train, cnt_test)

In [ ]:
cnt_pred = xgb_pred * .5 + lgbm_pred * .5

In [ ]:
submission = pd.read_csv("sampleSubmission.csv")

In [ ]:
submission['sentiment'] = cnt_pred

In [ ]:
submission.to_csv('countvect.csv', index = False)

Competition Late Leaderboard 상에서 0.93810으로 201위 랭크
***
#### Tfidf 적용

In [ ]:
tfidf = TfidfVectorizer(max_features = 5000, stop_words = 'english')

In [ ]:
tfidf_train = tfidf.fit_transform(train['review']).todense()

In [ ]:
tfidf_test = tfidf.transform(test['review']).todense()

In [ ]:
xgb_pred = get_model_proba(xgb, tfidf_train, tfidf_test)
lgbm_pred = get_model_proba(lgbm, tfidf_train, tfidf_test)

In [ ]:
tfidf_pred = xgb_pred * .5 + lgbm_pred * .5

In [ ]:
submission['sentiment'] = tfidf_pred

In [ ]:
submission.to_csv('tfidf.csv', index = False)

### 비지도 학습 기반 감성 분석

- 비지도 학습 기반은 Lexicon을 기반으로 한다. 한글 지원하는 Lexicon은 없다고 한다.
- 감성 사전을 구현해서 감성 지수(Polarity Score)라는 것을 만든다.

#### WordNet은 영어 어휘 사전으로 시맨틱 분석을 제공한다. 각각의 품사로 구성된 개별 단어를 Sysnet이라는 개념을 이용해 표현한다.
#### NLTK를 포함한 대표적인 감성 사전은 다음과 같다.
- SentiWordNet : 감성 단어 전용의 WordNet으로 Sysnet별로 3가지 감성 점수인 긍정 감성 지수, 부정 감성 지수, 객관성 지수를 할당한다. 문장별로 단어들의 긍정, 부정 감성 지수를 합산하여 최종 감성 지수를 계산하고 긍/부정을 결정한다.
- VADER : SNS에 대한 감성 분석을 제공하기 위한 패키지이다. 성능이 좋고 수행 시간이 빨라 대용량 텍스트 데이터에 잘 사용된다.
- Pattern : 예측 성능에서 가장 주목받는 패키지로 python 2.x 버전에서만 사용이 가능하다고 한다.
***
#### WordNet 실습

In [ ]:
import nltk
#nltk.download('all')

In [ ]:
from nltk.corpus import wordnet

In [ ]:
text = 'present'

present라는 단어의 sysnet을 반환한다.

In [ ]:
sysnet = wordnet.synsets(text)

In [ ]:
sysnet

의미, 품사, 의미 인덱스 순서를 가진다. for문으로 각 단어의 시맨틱적인 요소를 살펴보자.

In [ ]:
for s in sysnet :
    print(f'Sysnet name : {s.name()}')
    print(f'품사 : {s.lexname()}')
    print(f'정의 : {s.definition()}')
    print(f'부명제 : {s.lemma_names()}\n')

WordNet은 어휘 간의 관계를 유사도로 나타낼 수 있다.

In [ ]:
tree = wordnet.synset('tree.n.01')
cat = wordnet.synset('cat.n.01')
dog = wordnet.synset('dog.n.01')
flower = wordnet.synset('flower.n.01')
tiger = wordnet.synset('tiger.n.01')

In [ ]:
entities = [tree, cat, dog, flower, tiger]

In [ ]:
e_names = [e.name().split('.')[0] for e in entities]

In [ ]:
sims = []

In [ ]:
for e in entities :
    sim = [e.path_similarity(compare) for compare in entities]
    sims.append(sim)

In [ ]:
sim_df = pd.DataFrame(sims, columns = e_names, index = e_names)

In [ ]:
sim_df

#### SentiWordNet은 WordNet의 Sysnet과 유사한 Senti_Sysnet 클래스를 가지고 있다. 

In [ ]:
from nltk.corpus import sentiwordnet as swn

In [ ]:
ss = list(swn.senti_synsets('present'))

In [ ]:
ss

SentiSysnet은 감성 지수와 객관성 지수를 가지고 있다. 

In [ ]:
father = swn.senti_synset('father.n.01')

In [ ]:
print(f'father의 긍정 감성 지수 = {father.pos_score()}')
print(f'father의 부정 감성 지수 = {father.neg_score()}')
print(f'father의 객관성 지수 = {father.obj_score()}')

아버지를 뜻하는 father의 긍, 부정 지수는 0이고 객관성 지수는 1이다.

In [ ]:
nice = swn.senti_synset('nice.a.01')

In [ ]:
print(f'nice의 긍정 감성 지수 = {nice.pos_score()}')
print(f'nice의 부정 감성 지수 = {nice.neg_score()}')
print(f'nice의 객관성 지수 = {nice.obj_score()}')

nice는 긍정 감성 지수가 0.875이다.
***
WordNet과 SentiWordNet으로 영화 리뷰 감성 분석을 진행해보자.

In [ ]:
def transform_tag(x) :
    if x.startswith('J') :
        return wordnet.ADJ
    elif x.startswith('N') :
        return wordnet.NOUN
    elif x.startswith('R') :
        return wordnet.ADV
    elif x.startswith('V') :
        return wordnet.VERB

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [82]:
def get_sentiment(text) :
    
    sentiment = 0
    token_cnt = 0
    
    lemmatizer = WordNetLemmatizer()
    sentences = sent_tokenize(text)
    
    for sentence in sentences :
        
        tagged_sentence = pos_tag(word_tokenize(sentence))
        
        for word, tag in tagged_sentence :
            
            wn_tag = transform_tag(tag)
            if wn_tag not in (wordnet.NOUN, wordnet.ADJ, wordnet.ADV) :
                continue
            lemma = lemmatizer.lemmatize(word, pos = wn_tag)
            if not lemma :
                continue
                
            synsets = wordnet.synsets(lemma, pos = wn_tag)
            if not synsets :
                continue
            
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())

            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            token_cnt += 1
    if not token_cnt :
        return 0
    
    if sentiment >= 0 :
        return 1
    
    return 0

In [83]:
train['preds'] = train['review'].apply(get_sentiment)

In [91]:
acc = np.sum(train.sentiment == train.preds) / train.shape[0]

In [92]:
acc

0.66128

비지도 학습으로 예측한 모델의 정확도는 66%로 만족할만한 성능은 아니지만 레이블 없이 해냈다는 점에서 의의가 있다.
***
#### VADER를 이용한 감성 분석

In [94]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [95]:
analyzer = SentimentIntensityAnalyzer()

In [96]:
sent_score = analyzer.polarity_scores(train['review'][0])

In [97]:
sent_score

{'neg': 0.126, 'neu': 0.754, 'pos': 0.12, 'compound': -0.8696}

In [100]:
def vader_porlarity(text, threshold = 0.1) :
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(text)
    
    agg_score = score['compound']
    sentiment = 1 if agg_score >= threshold else 0
    
    return sentiment

In [101]:
train['vader_pred'] = train['review'].apply(vader_porlarity)

In [103]:
acc = np.sum(train.sentiment == train.vader_pred) / train.shape[0]

In [104]:
acc

0.69564

SentiWordNet 보다는 성능이 더 좋은 것을 알 수 있다.